In [ ]:
csv_path = 'C:/Users/vjosv/master/dataset/images/csv/combined_lokasjon_top_100_images200_seed1117_attempt2_with_loc.csv'


temp = 'C:/Users/vjosv/master/top_100_images_with_location_data.csv'
import time
import torch 
import torchvision
import torchvision.transforms as T
from PIL import Image
import matplotlib.pyplot as plt
import os
import torch
import pandas as pd
import numpy as np
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import torch.backends.cudnn as cudnn
import torch.nn as nn

import torch.nn.functional as F
import torch.nn.parallel
import torch.optim as optim
import torchvision
import torchvision.models as models
import sys

import ast
import shutil
from PIL import Image
from averagemeter import *
from models import *
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from torch.autograd import Variable
from torch.utils.data import sampler
from torchvision import datasets
from torchvision import transforms
import torchvision
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True


INPUT_SIZE = 224
NUM_CLASSES = 185
NUM_EPOCHS = 30
LEARNING_RATE = 1e-1
USE_CUDA = torch.cuda.is_available()
print(USE_CUDA)
best_prec1 = 0
classes = []

In [ ]:
import scipy
import numpy as np
import scipy.spatial as spatial
import matplotlib.pyplot as plt
import random 
import pandas as pd
import os
from mpl_toolkits.basemap import Basemap
from sklearn.datasets import fetch_species_distributions
from sklearn.neighbors import KernelDensity
import copy

from tqdm import tqdm

import torch
import torch.backends.cudnn as cudnn
import torch.nn as nn
import torch.nn.functional as F
import torch.nn.parallel
import torch.optim as optim
import torchvision
import torchvision.models as models

from pyproj import Proj, transform

from scipy.stats import norm
import warnings
from sklearn.neighbors import KNeighborsClassifier
import imblearn

import re

import utils


#copied from Kevin Barnes/kbarnes3: https://gist.github.com/kbarnes3/3fb7d353e9bdd3efccd5

import ctypes
import platform

ES_CONTINUOUS = 0x80000000
ES_SYSTEM_REQUIRED = 0x00000001


def _set_thread_execution(state):
    ctypes.windll.kernel32.SetThreadExecutionState(state)


def prevent_standby():
    if platform.system() == 'Windows':
        _set_thread_execution(ES_CONTINUOUS | ES_SYSTEM_REQUIRED)


def allow_standby():
    if platform.system() == 'Windows':
        _set_thread_execution(ES_CONTINUOUS)


def long_running(func):
    def inner(*args, **kwargs):
        prevent_standby()
        result = func(*args, **kwargs)
        allow_standby()
        return result
    return inner





import argparse
# !pip install opencv-python
import cv2
import json
import numpy as np
import os
# !pip install pandas
import pandas as pd
# !pip install scipy
import scipy.misc
import shutil
import time
import torch
import torch.backends.cudnn as cudnn
import torch.nn as nn
import torch.nn.functional as F
import torch.nn.parallel
import torch.optim as optim
import torchvision
import torchvision.models as models
import sys
    # caution: path[0] is reserved for script path (or '' in REPL)
# sys.path.insert(1, 'Deep-Leafsnap-master')

# !pip install scikit-image
# !pip install scikit-learn 
import utils

from scipy.stats import norm

from sklearn.neighbors import KNeighborsClassifier
# !pip install imblearn
import imblearn
from collections import Counter

from PIL import Image
from averagemeter import *
from models import *
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from torch.autograd import Variable
from torch.utils.data import sampler
from torchvision import datasets
from torchvision import transforms

import warnings

INPUT_SIZE = 224
NUM_CLASSES = 185
NUM_EPOCHS = 15*3
LEARNING_RATE = 1e-2
USE_CUDA = torch.cuda.is_available()
print(USE_CUDA)
best_prec1 = 0
classes = []


def validate(val_loader, model, criterion,epoch,save_output=False):
    # global output_aug_kde001, sample_pos, within_square, weight_tensor_kde001,zero_tensor_kde001,tt
    batch_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    top5 = AverageMeter()

    # losses_aug =  AverageMeter()

    
    conf = AverageMeter()
    class_correct = list(0. for i in range(185))
    class_total = list(0. for i in range(185))

    # switch to evaluate mode
    model.eval()

    end = time.time()
    for i, (input, target) in enumerate(val_loader):
        if USE_CUDA:
            input = input.cuda(non_blocking=True)
            target = target.cuda(non_blocking=True)
        input_var = torch.autograd.Variable(input, volatile=True)
        target_var = torch.autograd.Variable(target, volatile=True)

        # compute output
        output = model(input_var)

        loss = criterion(output, target)
        
        save_output_target_validate(model,output,target,epoch,i)

        
        prec1, prec5, conf1 = accuracy(output.data, target, topk=(1, 5))
        

        losses.update(loss.data.item(), input.size(0))
        
        top1.update(prec1.item(), input.size(0))
        top5.update(prec5.item(), input.size(0))

        conf.update(conf1,1)
        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        if i % 100 == 0:

            print('Test: [{0}/{1}]\n'.format(i, len(val_loader)))
            print('Time {batch_time.val:.3f} ({batch_time.avg:.3f})\n'.format(batch_time=batch_time))
            print('Loss {loss.val:.4f} ({loss.avg:.4f})\n'.format(loss=losses))
            print('Prec@1 {top1.val:.3f} ({top1.avg:.3f})\n'.format(top1=top1))
            print('Prec@5 {top5.val:.3f} ({top5.avg:.3f})\n'.format(top5=top5))

            print(' * Prec@1 {top1.avg:.3f} Prec@5 {top5.avg:.3f}'
              .format(top1=top1, top5=top5))
        


    return top1.avg

def train(train_loader, model, criterion, optimizer, epoch):

    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    top5 = AverageMeter()

    # switch to train mode
    model.train()

    end = time.time()
    for i, (input, target) in enumerate(train_loader):
        # measure data loading time
        if USE_CUDA:
            input = input.cuda(non_blocking=True)
            target = target.cuda(non_blocking=True)

        data_time.update(time.time() - end)

        input_var = torch.autograd.Variable(input)
        target_var = torch.autograd.Variable(target)

        # compute output
        output = model(input_var)
        # print(output)

        loss = criterion(output, target_var)
        # print(loss)
        # if i >10:
        #     break
        
        # save_output_target(model,output,target_var,epoch,i)
        # measure accuracy and record loss
        prec1, prec5,conf = accuracy(output.data, target, topk=(1, 5))
        losses.update(loss.data.item(), input.size(0))
        top1.update(prec1.item(), input.size(0))
        top5.update(prec5.item(), input.size(0))
        
        save_output_target_train(model,output,target_var,epoch,i)
        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        if i % 100 == 0:
            print('Epoch: [{0}][{1}/{2}]\t'
                  '\Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Data {data_time.val:.3f} ({data_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                  'Prec@1 {top1.val:.3f} ({top1.avg:.3f})\t'
                  'Prec@5 {top5.val:.3f} ({top5.avg:.3f})'.format(
                      epoch, i, len(train_loader), batch_time=batch_time,
                      data_time=data_time, loss=losses, top1=top1, top5=top5))


In [ ]:


# dat_images = pd.read_csv('C:/Users/vjosv/master/dataset/images/csv/combined_lokasjon_top_100_images200_seed1117.csv')

# dat = pd.read_csv('C:/Users/vjosv/master/dataset/images/csv/second_try_lokasjon_top_100_csv.csv')                         

# dat_images_id = list(dat_images['Id'])                        
                         

# dat_images['points']= [[i,j] for i, j in zip(dat_images.Østkoordinat, dat_images.Nordkoordinat)]

# dat['points']= [[i,j] for i, j in zip(dat.Østkoordinat, dat.Nordkoordinat)]


# dat = dat[~dat['Id'].isin(dat_images_id)]

# points = dat[['Østkoordinat','Nordkoordinat']].to_numpy()
# # category = dat['target'].to_numpy()



# point_tree = spatial.cKDTree(points)



traindir = os.path.join('dataset', 'train')
testdir = os.path.join('dataset', 'test')
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])
data_train = datasets.ImageFolder(traindir, transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    normalize]))
data_val = datasets.ImageFolder(testdir, transforms.Compose([
    transforms.ToTensor(),
    normalize]))
classes = data_train.classes

data_train = torch.utils.data.DataLoader(data_train, batch_size=64, shuffle=True, num_workers=2)
data_val = torch.utils.data.DataLoader(data_val, batch_size=64, shuffle=False, num_workers=2)
def save_output_target_validate(model,output,target,epoch,batch):
        torch.save(output,f'saved_output/{model.name}_validating_output/output_e{epoch}_b{batch}')
        torch.save(target,f'saved_output/{model.name}_validating_output/target_b{batch}')
        
def save_output_target_train(model,output,target,epoch,babtch):
        torch.save(output,f'saved_output/{model.name}_training_output/output_e{epoch}_b{batch}')

        torch.save(target,f'saved_output/{model.name}_training_output/target_e_{epoch}_b{batch}')


In [ ]:
model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet18', pretrained=True)
model.name = "resnet18_pretrained2911"
model.fc=nn.Linear(model.fc.in_features, 185)

if not os.path.isdir(f'saved_output/{model.name}_validating_output'):
    os.mkdir(f'saved_output/{model.name}_validating_output')
if not os.path.isdir(f'saved_output/{model.name}_training_output'):
    os.mkdir(f'saved_output/{model.name}_training_output')

if USE_CUDA:
    model.cuda()
    
if USE_CUDA:
    model.cuda()
train_model(model,LEARNING_RATE = 1e-2,NUM_EPOCHS = 15)
train_model(model,LEARNING_RATE = 1e-3,NUM_EPOCHS = 30)
train_model(model,LEARNING_RATE = 1e-4,NUM_EPOCHS = 45)

model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet34', pretrained=True)
model.fc=nn.Linear(model.fc.in_features, 185)
model.name = "resnet34_pretrained2911"

if not os.path.isdir(f'saved_output/{model.name}_validating_output'):
    os.mkdir(f'saved_output/{model.name}_validating_output')
if not os.path.isdir(f'saved_output/{model.name}_training_output'):
    os.mkdir(f'saved_output/{model.name}_training_output')

if USE_CUDA:
    model.cuda()
train_model(model,LEARNING_RATE = 1e-2,NUM_EPOCHS = 15)
train_model(model,LEARNING_RATE = 1e-3,NUM_EPOCHS = 30)
train_model(model,LEARNING_RATE = 1e-4,NUM_EPOCHS = 45)


model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet50', pretrained=True)
model.name = "resnet50_pretrained2911"
model.fc=nn.Linear(model.fc.in_features, 185)


if not os.path.isdir(f'saved_output/{model.name}_validating_output'):
    os.mkdir(f'saved_output/{model.name}_validating_output')
if not os.path.isdir(f'saved_output/{model.name}_training_output'):
    os.mkdir(f'saved_output/{model.name}_training_output')

if USE_CUDA:
    model.cuda()
train_model(model,LEARNING_RATE = 1e-2,NUM_EPOCHS = 15)
train_model(model,LEARNING_RATE = 1e-3,NUM_EPOCHS = 30)
train_model(model,LEARNING_RATE = 1e-4,NUM_EPOCHS = 45)


model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet101', pretrained=True)
model.name = "resnet101_pretrained2911"
model.fc=nn.Linear(model.fc.in_features, 185)


if not os.path.isdir(f'saved_output/{model.name}_validating_output'):
    os.mkdir(f'saved_output/{model.name}_validating_output')
if not os.path.isdir(f'saved_output/{model.name}_training_output'):
    os.mkdir(f'saved_output/{model.name}_training_output')

if USE_CUDA:
    model.cuda()
train_model(model,LEARNING_RATE = 1e-2,NUM_EPOCHS = 15)
train_model(model,LEARNING_RATE = 1e-3,NUM_EPOCHS = 30)
train_model(model,LEARNING_RATE = 1e-4,NUM_EPOCHS = 45)


model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet152', pretrained=True)
model.name = "resnet152_pretrained2911"
model.fc=nn.Linear(model.fc.in_features, 185)


if not os.path.isdir(f'saved_output/{model.name}_validating_output'):
    os.mkdir(f'saved_output/{model.name}_validating_output')
if not os.path.isdir(f'saved_output/{model.name}_training_output'):
    os.mkdir(f'saved_output/{model.name}_training_output')

if USE_CUDA:
    model.cuda()
train_model(model,LEARNING_RATE = 1e-2,NUM_EPOCHS = 15)
train_model(model,LEARNING_RATE = 1e-3,NUM_EPOCHS = 30)
train_model(model,LEARNING_RATE = 1e-4,NUM_EPOCHS = 45)



### Create geo samples by running geo_functions_leafsnap.ipynb

### test geo on leafsnap


In [ ]:






def accuracy(output, target, topk=(1,)):
    """Computes the precision@k for the specified values of k"""
    maxk = max(topk)
    
    batch_size = target.size(0)

    _, pred = output.topk(maxk, 1, True, True)
    pred_t = pred.t()

    correct = pred_t.eq(target.view(1, -1).expand_as(pred_t))

    nb_classes = 185

    confusion_matrix = torch.zeros(nb_classes, nb_classes)
    _, preds = torch.max(output, 1)
    
    for t, p in zip(target.view(-1), preds.view(-1)):
        confusion_matrix[t.long(), p.long()] += 1

    # print(confusion_matrix)
    
    
    # print(confusion_matrix.diag()/confusion_matrix.sum(1))
        
    res = []
    for k in topk:
    	correct_k = torch.sum(torch.reshape(correct[:k],(-1,) ).float()) #.double().sum(0)
    	res.append(correct_k.mul_(100.0 / batch_size))
    res.append(confusion_matrix)
    return res


In [ ]:
# knn

model_name = "vgg19_pretrained"

list_outputs = os.listdir(f'saved_output/geo_functions/')


In [ ]:
kde_outputs = []
for lo in list_outputs:
    if lo[:3] == 'kde':
        kde_outputs.append(lo)

In [ ]:
# kde_outputs[0]

side_len = [1000,2000,3000,4000]
bandwidth = [500,1000,1500, 2000,3000,4000]
configuration =[0,1,2]

def augment_output_add(output, augment, add):
    output_ = torch.nn.functional.softmax(output, dim=1)
    augment = augment/augment.sum(dim=1, keepdim=True)
    output_aug = output_ + add*augment
    output_aug = output_aug * 1/output_aug.sum(dim=1, keepdim=True)
    
    return torch.log(output_aug)


In [ ]:
model_name2 ="resnet101_pretrained"

add = 1.7
max_epoch = 44

avg_prec1 =[]
avg_prec1_temp =[]
for e in range(max_epoch+1):#max_e+1):
    for b in range(max_b+1):
        output = torch.load(f'saved_output/{model_name2}_validating_output/output_e{e}_b{b}')
        target = torch.load(f'saved_output/{model_name2}_validating_output/target_b{b}')


        prec1, prec5, conf1 = accuracy(output.data, target, topk=(1, 5))
        avg_prec1_temp.append(prec1.cpu())
    avg_prec1.append(np.average(avg_prec1_temp))
    avg_prec1_temp =[]


best_epoch = np.argmax(avg_prec1)
print(best_epoch)
print(avg_prec1[best_epoch])


In [ ]:
from tqdm import tqdm

prec1_list =[]
prec1_avg_list_aug = []
prec1_avg_list_temp = []



prec5_list = []
prec5_avg_list_aug = []
prec5_avg_list_temp = []




side_len = [1000,2000,3000,4000]
bandwidth = [500,1000,1500, 2000,3000,4000]
configuration =[0,1,2]

model_name ="resnet18_pretrained"

add = 1.7
max_epoch = 44
max_b = 92
avg_prec1 =[]
avg_prec1_temp =[]
avg_prec5 =[]
avg_prec5_temp =[]
for e in range(max_epoch+1):#max_e+1):
    for b in range(max_b+1):
        output = torch.load(f'saved_output/{model_name}_validating_output/output_e{e}_b{b}')
        target = torch.load(f'saved_output/{model_name}_validating_output/target_b{b}')


        prec1, prec5, conf1 = accuracy(output.data, target, topk=(1, 5))
        avg_prec1_temp.append(prec1.cpu())
        avg_prec5_temp.append(prec5.cpu())
    avg_prec1.append(np.average(avg_prec1_temp))
    avg_prec1_temp =[]
    avg_prec5.append(np.average(avg_prec5_temp))
    avg_prec5_temp =[]


best_epoch = np.argmax(avg_prec1)
top1 = avg_prec1[best_epoch]
top5=avg_prec5[best_epoch]
print(best_epoch ,top1, top5)

           # print(dict_df)
dict_example = {'Side Length': [],
            'Bandwidth': [],
            'add':[],
             'epoch': [],
             'prec1$_a$': [],
            'prec1':[],
            'prec5':[],
            'prec5$_a$':[],
           }

dict_df = dict_example #[copy.deepcopy(dict_example) for i in range(4)]


           

def augment_output_add(output, augment, add):
    output_ = torch.nn.functional.softmax(output, dim=1)
    augment = augment/augment.sum(dim=1, keepdim=True)
    output_aug = output_ + add*augment
    output_aug = output_aug/output_aug.sum(dim=1,keepdim=True)
    
    return output_aug


max_e
add_list = [0.1,0.2,0.4,0.6,0.8,1]
add = 0.2
for add in tqdm(add_list):
    
    for sl in side_len:
        for bw in bandwidth:

            prec1_list = []
            prec1_list_aug = []
            prec1_avg_list_aug = []
            prec1_avg_list_aug_temp = []

            prec5_list = []
            prec5_list_aug = []
            prec5_avg_list_aug = []
            prec5_avg_list_aug_temp = []

            prec5_avg_list = []
            prec5_avg_list_temp = []

            prec1_avg_list = []
            prec1_avg_list_temp=[]

            prec1_avg_list = []

            e = best_epoch

            for b in range(max_b+1):
                output = torch.load(f'saved_output/{model_name}_validating_output/output_e{e}_b{b}')
                target = torch.load(f'saved_output/{model_name}_validating_output/target_b{b}')

                tensor_list = []


                load_tensor0 =torch.load( f'saved_output/geo_functions/kde_side_len{sl}m_bandwitdth{bw}_batch{b}_cofiguration{0}')
                load_tensor1 =torch.load( f'saved_output/geo_functions/kde_side_len{sl}m_bandwitdth{bw}_batch{b}_cofiguration{1}')
                load_tensor2 =torch.load( f'saved_output/geo_functions/kde_side_len{sl}m_bandwitdth{bw}_batch{b}_cofiguration{2}')

                try:
                    load_tensor0 = load_tensor0*1/torch.sum(load_tensor0)

                except:
                    print('f')
                    pass
                try:
                    load_tensor1 = load_tensor1*1/torch.sum(load_tensor1)
                except:
                    print('f')
                    pass
                try:
                    load_tensor2 = load_tensor2*1/torch.sum(load_tensor2)
                except:
                    print('f')
                    pass


                output0 = augment_output_add(output,load_tensor0.cuda(),add)
                output1 = augment_output_add(output,load_tensor1.cuda(),add)
                output2 = augment_output_add(output,load_tensor2.cuda(),add)


                prec10, prec50, conf1 = accuracy(output0.data, target, topk=(1, 5))
                prec11, prec51, conf1 = accuracy(output1.data, target, topk=(1, 5))
                prec12, prec52, conf1 = accuracy(output2.data, target, topk=(1, 5))

                prec1, prec5, conf1 = accuracy(output.data, target, topk=(1, 5))
                # print(output0)
                prec1_avg = (prec10+prec11+prec12)/3
                # print(prec1_avg)
                # print('...........')
                prec5_avg = (prec50+prec51+prec52)/3






                prec1_list.append(prec1_avg.cpu())
                prec1_avg_list_temp.append(prec1_avg.cpu())

                prec5_list.append(prec5_avg.cpu())
                prec5_avg_list_temp.append(prec5_avg.cpu())



            print(np.average(prec1_avg_list_temp),sl,bw,add)
            prec1_avg_list_aug.append(np.average((prec1_avg_list_temp)))


            prec5_avg_list_aug.append(np.average((prec5_avg_list_temp)))




       # print(dict_df)
            # print(i)
            # print(dict_df[i])
            dict_df['Side Length'].append(sl)
            dict_df['Bandwidth'].append(bw)
            dict_df['add'].append(add)
            dict_df['epoch'].append(best_epoch)
            dict_df['prec1$_a$'].append(np.average(prec1_avg_list_temp))

            dict_df['prec1'].append(top1)
            dict_df['prec5'].append(top5)
            dict_df['prec5$_a$'].append(np.average(prec5_avg_list_temp))


            prec5_avg_list_temp = []
            prec1_avg_list_temp = []

            print(dict_df)

   

dict_df['prec1$_a$-prec1']= [dict_df['prec1$_a$'][i] - dict_df['prec1'][i] for i in range(len(dict_df['prec1']))]
dict_df['prec5$_a$-prec5']= [dict_df['prec5$_a$'][i] - dict_df['prec5'][i] for i in range(len(dict_df['prec1']))]


In [ ]:
# dict_df                                 
b = pd.DataFrame.from_dict(dict_df)
# a

print(b[['add','Side Length', 'Bandwidth','epoch','prec1$_a$','prec5$_a$','prec1$_a$-prec1','prec5$_a$-prec5' ]].to_latex(index=False))
# print(a.to_latex(index=False))
# dict_df[20]

In [ ]:
#KNN
from tqdm import tqdm
import copy

prec1_list =[]
prec1_avg_list_aug = []
prec1_avg_list_temp = []



prec5_list = []
prec5_avg_list_aug = []
prec5_avg_list_temp = []





add = 1.7

configurations = [0,1,2]

n_list = [25,50,75,100,200,500,1000,1500,2000,2500,3000]



temp_list = []
prec1_l = []
model_2 ="resnet18_pretrained"

models_ =["resnet18_pretrained","resnet34_pretrained","resnet50_pretrained","resnet101_pretrained","vgg16_pretrained","vgg19_pretrained"]
max_e_knn= 44
max_b = 92
prec1_avg_list = []
prec1_avg_list_temp =[]
for e in range(max_e+1):
    for b in range(max_b+1):
        output = torch.load(f'saved_output/{model_2}_validating_output/output_e{e}_b{b}')
        target = torch.load(f'saved_output/{model_2}_validating_output/target_b{b}')


        prec10, prec50, conf10 = accuracy(output.data, target, topk=(1, 5))

        # print(prec10)
        prec1_avg_list_temp.append(prec10.cpu())


    prec1_avg_list.append(np.average((prec1_avg_list_temp)))

    prec1_avg_list_temp = []


best_epoch = np.argmax(prec1_avg_list)
print(best_epoch)
max_original = prec1_avg_list[best_epoch]
print(max_original)

#              }
dict_example = {'k': [],
                'add':[],
             'best epoch$_a$': [],
             'prec1$_a$': [],
            'prec5$_a$':[],
            'prec1$_{max}$':[],
            'prec5$_{max}$':[],
           }

dict_df =  dict_example #[copy.deepcopy(dict_example) for i in range(1)]


def augment_output_add(output, augment, add):
    output_ = torch.nn.functional.softmax(output, dim=1)
    augment = augment/augment.sum(dim=1, keepdim=True)
    output_aug = output_ + add*augment
    output_aug = output_aug/output_aug.sum(dim=1,keepdim=True)
    
    return output_aug

print(best_epoch)
e = max_e
max_value = 0
max_e_knn= 44
max_add = 0
add_list = [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1]#,1.1,1.2,1.3,1.4,1.5,1.6,1.7,1.8,1.9,2]
# best add:
add = add_list[5]
print('add:',add)


for add in tqdm(add_list):
    for n in n_list:
        prec1_list = []
        prec1_avg_list_aug = []
        prec1_avg_list_temp = []

        prec5_list = []
        prec5_avg_list_aug = []
        prec5_avg_list_temp = []
        prec1_list_original =[]
        prec1_list_original_temp = []
        prec5_list_original =[]
        prec5_list_original_temp = []


        e=best_epoch
        for b in range(max_b+1):
            output = torch.load(f'saved_output/{model_2}_validating_output/output_e{e}_b{b}')
            target = torch.load(f'saved_output/{model_2}_validating_output/target_b{b}')

            tensor_list = []

            load_tensor0 =torch.load( f'saved_output/geo_functions/knn_smote_n_{n}_batch{b}_cofiguration{0}')
            load_tensor1 =torch.load( f'saved_output/geo_functions/knn_smote_n_{n}_batch{b}_cofiguration{1}')
            load_tensor2 =torch.load( f'saved_output/geo_functions/knn_smote_n_{n}_batch{b}_cofiguration{2}')



            output0 = augment_output_add(output,load_tensor0.cuda(),add)
            output1 = augment_output_add(output,load_tensor1.cuda(),add)
            output2 = augment_output_add(output,load_tensor2.cuda(),add)


            prec10, prec50, conf1 = accuracy(output0.data, target, topk=(1, 5))
            prec11, prec51, conf1 = accuracy(output1.data, target, topk=(1, 5))
            prec12, prec52, conf1 = accuracy(output2.data, target, topk=(1, 5))

            prec1, prec5, conf1 = accuracy(output.data, target, topk=(1, 5))

            prec1_avg = (prec10+prec11+prec12)/3

            prec5_avg = (prec50+prec51+prec52)/3

            prec1_list.append(prec1_avg.cpu())
            prec1_avg_list_temp.append(prec1_avg.cpu())

            prec5_list.append(prec5_avg.cpu())
            prec5_avg_list_temp.append(prec5_avg.cpu())

            prec1_list_original_temp.append(prec1.cpu())
            prec5_list_original_temp.append(prec5.cpu())

        print(np.average(prec1_avg_list_temp),max_original,n,add)

        prec1_avg_list_aug.append(np.average(prec1_avg_list_temp))

        prec1_list_original.append(np.average((prec1_list_original_temp)))

        prec5_list_original.append(np.average(prec5_list_original_temp))





        dict_df['k'].append(n)
        dict_df['add'].append(add)
        dict_df['best epoch$_a$'].append(best_epoch)
        dict_df['prec1$_a$'].append(np.average(prec1_avg_list_temp))
        dict_df['prec1$_{max}$'].append(np.average(prec1_list_original_temp))
        dict_df['prec5$_a$'].append(np.average(prec5_avg_list_temp))
        dict_df['prec5$_{max}$'].append(np.average(prec5_list_original_temp))
        print(prec1_avg_list_aug)
        print(dict_df)

        prec5_avg_list_temp = []
        prec1_avg_list_temp = []
        prec5_list_original_temp = []
        prec1_list_original_temp = []



In [ ]:
dict_df['prec1$_a$-prec1$_{max}$']= [dict_df['prec1$_a$'][i] - dict_df['prec1$_{max}$'][i] for i in range(len(dict_df['prec5$_{max}$']))]
dict_df['prec5$_a$-prec5$_{max}$']= [dict_df['prec5$_a$'][i] - dict_df['prec5$_{max}$'][i] for i in range(len(dict_df['prec5$_{max}$']))]
# dict_df   


b = pd.DataFrame.from_dict(dict_df)
# a
# print(model_2)
print(b[['add','k', 'best epoch$_a$','prec1$_a$','prec5$_a$','prec1$_a$-prec1$_{max}$','prec5$_a$-prec5$_{max}$' ]].to_latex(index=False))
# print(a.to_latex(index=False))



In [ ]:
for i in [3]:
    a = pd.DataFrame.from_dict(dict_df[i])
    print(a.to_latex())
    
    a.to_csv(f'knn_results_configuration{i}.csv')

#### In Area weight

In [ ]:
sample_radius = [50,100,200,300,400,500,600,700,800,900,1000,1100,1200,1300,1400,1500]
configuration = [i for i in range(20)]
# configurations = [0]*

In [ ]:
import copy

model_2 = 'resnet18_pretrained'
# model_2 = "vgg19_pretrained"
max_e = 44
max_b = 92
def augment_output_multiply(output,augment,mul):
    output_aug = torch.nn.functional.softmax(output, dim=1)
    # output_aug = torch.nn.functional.normalize(output_aug*augment)
    output_aug = output_aug +mul*augment
    output_aug = torch.nn.functional.normalize(output_aug)
    output_aug = torch.log(output_aug)
    
    
    return output_aug

configurations = [i for i in range(20)]
prec1_list = [[],[],[],[]]
prec1_avg_list_aug = [[],[],[],[]]
prec1_avg_list_temp = [[],[],[],[]]

prec1_avg_list = []

prec5_list = [[],[],[],[]]
prec5_avg_list_aug = [[],[],[],[]]
prec5_avg_list_temp = [[],[],[],[]]


add = 1.7

dict_example = {'Sample Radius': [],
             'best epoch$_a$': [],
            'prec1$_a$': [], 
            'prec1$_{max}$':[],
            'prec5$_{max}$':[],
            'prec5$_a$':[],
            f'prec1$_a$ at epoch {best_epoch}':[],
            f'prec5$_a$ at epoch {best_epoch}':[],
             }


dict_df = [copy.deepcopy(dict_example) for i in range(21)]



add_list = [10]


def augment_output_add(output, augment, add):
    output_ = torch.nn.functional.softmax(output, dim=1)
    augment = augment/augment.sum(dim=1, keepdim=True)
    output_aug = output_*augment
    output_aug = output_aug/output_aug.sum(dim=1,keepdim=True)
    
    return output_aug #torch.log(output_aug)


prec1_avg_list = []
prec1_avg_list_temp = []
prec5_avg_list = []
prec5_avg_list_temp = []

for e in range(max_e+1):

    for b in range(max_b+1):
        output = torch.load(f'saved_output/{model_2}_validating_output/output_e{e}_b{b}')
        target = torch.load(f'saved_output/{model_2}_validating_output/target_b{b}')

        # tensor_avg =(tensor_list[0]+tensor_list[1]+tensor_list[2])/3

        # output1 = augment_output_add(output,load_tensor.cuda(),add)


        # prec1, prec5, conf1 = accuracy(output1.data, target, topk=(1, 5))
        prec10, prec50, conf10 = accuracy(output.data, target, topk=(1, 5))

        # print(prec10)
        prec1_avg_list_temp.append(prec10.cpu())
        prec5_avg_list_temp.append(prec50.cpu())



    prec1_avg_list.append(np.average((prec1_avg_list_temp)))
    prec5_avg_list.append(np.average((prec5_avg_list_temp)))
    prec1_avg_list_temp = []
    prec5_avg_list_temp = []

best_epoch = np.argmax(prec1_avg_list)
print(best_epoch)
best_value1 = prec1_avg_list[best_epoch]
print(best_value1)
best_value5 = prec5_avg_list[best_epoch]


for e in range(max_e+1):
    for b in range(max_b+1):
        output = torch.load(f'saved_output/{model_2}_validating_output/output_e{e}_b{b}')
        target = torch.load(f'saved_output/{model_2}_validating_output/target_b{b}')


        prec10, prec50, conf10 = accuracy(output.data, target, topk=(1, 5))

        # print(prec10)
        prec1_avg_list_temp.append(prec10.cpu())


    prec1_avg_list.append(np.average((prec1_avg_list_temp)))

    prec1_avg_list_temp = []


best_epoch = np.argmax(prec1_avg_list)
print(best_epoch)
max_original = prec1_avg_list[best_epoch]
print(max_original)

e = max_e
add_list = [0.1,0.25,0.5,0.75,1.1,1.2,1.3,1.4,1.5]
models_ =["resnet18_pretrained","resnet34_pretrained","resnet50_pretrained","resnet101_pretrained","vgg16_new","vgg19_new"]

for model_2 in models_[0:1]:
    
    max_e = 27



    dict_example = {'Sample Radius': [],
             'best epoch$_a$': [],
            'add':[],
            'prec1$_a$': [], 
            'prec1$_{max}$':[],
            'prec5$_{max}$':[],
            'prec5$_a$':[],
             'prec1$_a$-prec1$_{max}$':[],
             'prec5$_a$-prec5$_{max}$' : [],
             }


    dict_df = dict_example #[copy.deepcopy(dict_example) for i in range(21)]


    
    for add in add_list:
        for i,sr in enumerate([10,20,30,40,50,100,200,300,400,500,1000,2000]):#sample_radius[:1]:
            prec1_list = []
            prec1_list_aug = []
            prec1_avg_list_aug = []
            prec1_avg_list_aug_temp = []

            prec5_list = []
            prec5_list_aug = []
            prec5_avg_list_aug = []
            prec5_avg_list_aug_temp = []

            # prec5_avg_list = []
            # prec5_avg_list_temp = []

            # prec1_avg_list = []
            prec1_avg_list_temp=[]

            # prec1_avg_list = []

            e= best_epoch
            for b in range(max_b+1):
                output = torch.load(f'saved_output/{model_2}_validating_output/output_e{e}_b{b}')
                target = torch.load(f'saved_output/{model_2}_validating_output/target_b{b}')

                tensor_list = []


                load_tensors = [torch.load( f'saved_output/geo_functions/sampled_{sr}m_batch{b}_cofiguration{i}') for i in configuration]
                # load_tensor = torch.load( f'saved_output/{model_name}_validating_output/sampled_{sr}m_batch{b}_cofiguration{i}')
                # print(load_tensors)
                output_augs = [augment_output_multiply(output.cuda(),load_tensor.cuda(),add) for load_tensor in load_tensors]

                # output_aug = augment_output_multiply(output,load_tensor.cuda(),add)
                # print(output_aug)
                precs =[(accuracy(output_aug.data, target, topk=(1, 5))) for output_aug in output_augs]

                prec10, prec50, conf10 = accuracy(output.data, target, topk=(1, 5))
                prec1_avg_list_temp.append(prec10.cpu())
                prec5_avg_list_temp.append(prec50.cpu())

                prec1s = 0
                prec5s = 0

                for p15 in precs:
     
                    prec1s+= p15[0]
                    prec5s+= p15[1]
                # print(output0)
                prec1_avg = prec1s/len(precs)
                # print(prec1_avg)
                # print('...........')
                prec5_avg = prec5s/len(precs)





                # print(prec1_avg)
                prec1_list.append(prec1_avg.cpu())
                prec1_avg_list_aug_temp.append(prec1_avg.cpu())

                prec5_list.append(prec5_avg.cpu())
                prec5_avg_list_aug_temp.append(prec5_avg.cpu())


    
            dict_df['Sample Radius'].append(sr)

            # dict_df[i]['best epoch$_a$'].append(np.argmax(prec1_avg_list_aug[i]))
            dict_df['best epoch$_a$'].append(e)
            dict_df['add'].append(add)   
            dict_df['prec1$_a$'].append(np.average(prec1_list))
            dict_df['prec1$_{max}$'].append(best_value1)
            dict_df['prec5$_{max}$'].append(best_value5)
            dict_df['prec5$_a$'].append(np.average(prec5_list))

 
            dict_df['prec1$_a$-prec1$_{max}$'].append(np.average(prec1_list) -best_value1)
            dict_df['prec5$_a$-prec5$_{max}$'].append(np.average(prec5_list)-best_value5)
            print(dict_df)
            prec5_avg_list_aug_temp = []
            prec1_avg_list_temp = []
            prec1_avg_list_aug_temp = []
                               
    b = pd.DataFrame.from_dict(dict_df )
    # a
    print(model_2)
    print(b[['Sample Radius', 'best epoch$_a$','prec1$_a$','prec5$_a$','prec1$_a$-prec1$_{max}$','prec5$_a$-prec5$_{max}$' ]].to_latex(index=False))
    # print(a.to_latex(index=False))
    # dict_df[20]
    b.to_csv(f'sample_radius{model_2}_results.csv')

In [ ]:
b = pd.DataFrame.from_dict(dict_df )
b
print(b[['Sample Radius', 'add','best epoch$_a$','prec1$_a$','prec5$_a$','prec1$_a$-prec1$_{max}$','prec5$_a$-prec5$_{max}$' ]].to_latex(index=False))